In [1]:
import os
from pathlib import Path

import pandas as pd
import pickle
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from pydantic import BaseModel
from enum import Enum, auto
from tqdm import tqdm

from src.constants import PROJECT_ROOT_PATH
from src.ocr import EngineType
from src.region_processing import process_region_segments_ocr, process_region_segments_checkbox

pd.set_option('display.max_rows', 20)

In [2]:
class SegmentType(str, Enum):
    OCR = "ocr"
    CHECKBOX = "checkbox"


class ProcessSegmentParams(BaseModel):
    base_path: Path
    segment_type: SegmentType
    skip_folders: list[str]
    default_fillna: str = 'SCAN ERROR / NO PREDICTION'


In [3]:
all_data_path = PROJECT_ROOT_PATH / "data/WPA_Bunker_Hill_Curated.xlsx"
all_data = pd.read_excel(all_data_path)

In [4]:
classifier = PROJECT_ROOT_PATH / 'static/checkbox_model_2.pkl'

segment_params = [
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/heating", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/race", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/rent", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/roomers", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/business_units", segment_type=SegmentType.CHECKBOX, skip_folders=["num"], default_fillna="yes"),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/business_units", segment_type=SegmentType.OCR, skip_folders=["none"], default_fillna="none"),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/basement", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/bath", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/condition", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/converted", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/cooking", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/duration", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/encumbrance", segment_type=SegmentType.CHECKBOX, skip_folders=[], default_fillna=""),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/ext_material", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/extra_families", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/garage", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/included_in_rent", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/lighting", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/major_structures", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/number_and_age", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/occupancy", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    # ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/office", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/refrig", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/stories", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/structure", segment_type=SegmentType.CHECKBOX, skip_folders=[], default_fillna="other"),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/structure_text", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/toilets", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/total_rooms", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/value", segment_type=SegmentType.OCR, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/water", segment_type=SegmentType.CHECKBOX, skip_folders=[]),
    ProcessSegmentParams(base_path=PROJECT_ROOT_PATH / "data/06_Segments/year_built", segment_type=SegmentType.OCR, skip_folders=[]),
]

In [5]:
def merge_into_all_data_and_save(all_data, df):
    df.index = df.index.astype(str)
    extracted_ids = df.index.str.extract('_(\d+)\.\w+$')[0]
    match_id_series = pd.Series(data=extracted_ids.values, index=df.index)
    match_id_series.dropna(inplace=True)
    df['match_ids'] = match_id_series.astype(int)
    
    print(df.head())

    merged_df = pd.merge(all_data, df, on='match_ids', how='left')
    return merged_df


extracted_ids = all_data['CONTENTdm file path'].str.extract('\/(\d+)\.jp2$')[0]
match_id_series = pd.Series(data=extracted_ids.values, index=all_data.index)
match_id_series.dropna(inplace=True)
all_data['match_ids'] = match_id_series.astype(int)

segment_results = {}
for params in tqdm(segment_params):
    if params.segment_type == SegmentType.CHECKBOX:
        result_df = process_region_segments_checkbox(params.base_path, classifier, skip_folders=params.skip_folders, default_fillna=params.default_fillna)
    elif params.segment_type == SegmentType.OCR:
        result_df = process_region_segments_ocr(params.base_path, EngineType.PADDLEOCR, skip_folders=params.skip_folders, default_fillna=params.default_fillna)
    segment_results[params.base_path.name] = result_df
    print(f"{params.base_path.name} segment processed data head:")
    print(result_df.head())

    all_data = merge_into_all_data_and_save(all_data, result_df)
    
    output_path = PROJECT_ROOT_PATH / f"data/WPA_Bunker_Hill_Curated_Updated.xlsx"
    all_data.to_excel(output_path, index=False)
    print(f"Data saved to {output_path}")

all_data.drop(columns=['match_ids'], inplace=True)
output_path = PROJECT_ROOT_PATH / f"data/WPA_Bunker_Hill_Curated_Final.xlsx"
all_data.to_excel(output_path, index=False)
print(f"Final data saved to {output_path}")

    


  0%|          | 0/30 [00:00<?, ?it/s]

Processing images: 100%|██████████| 8716/8716 [00:06<00:00, 1352.99it/s]


heating segment processed data head:
                                           heating
p15799coll8_75836.jpeg     cent_steam_or_hot_water
p15799coll8_79403.jpeg                        none
p15799coll8_13025.jpeg                        none
p15799coll8_92298.jpeg     cent_steam_or_hot_water
p15799coll8_44403.jpg   SCAN ERROR / NO PREDICTION
                                           heating  match_ids
p15799coll8_75836.jpeg     cent_steam_or_hot_water      75836
p15799coll8_79403.jpeg                        none      79403
p15799coll8_13025.jpeg                        none      13025
p15799coll8_92298.jpeg     cent_steam_or_hot_water      92298
p15799coll8_44403.jpg   SCAN ERROR / NO PREDICTION      44403


  3%|▎         | 1/30 [00:10<05:17, 10.95s/it]

Data saved to /Users/xieewenz/BunkerHill-CardScanToData/data/WPA_Bunker_Hill_Curated_Updated.xlsx


...

Processing OCR: 100%|██████████| 2179/2179 [08:53<00:00,  4.08it/s]


year_built segment processed data head:
                       year_built.num
p15799coll8_3710.jpeg            1905
p15799coll8_14426.jpeg           1895
p15799coll8_75836.jpeg             90
p15799coll8_30529.jpeg           1914
p15799coll8_49866.jpg             190
                       year_built.num  match_ids
p15799coll8_3710.jpeg            1905       3710
p15799coll8_14426.jpeg           1895      14426
p15799coll8_75836.jpeg             90      75836
p15799coll8_30529.jpeg           1914      30529
p15799coll8_49866.jpg             190      49866


100%|██████████| 30/30 [5:13:50<00:00, 627.69s/it]

Data saved to /Users/xieewenz/BunkerHill-CardScanToData/data/WPA_Bunker_Hill_Curated_Updated.xlsx


Final data saved to /Users/xieewenz/BunkerHill-CardScanToData/data/WPA_Bunker_Hill_Curated_Final.xlsx
